## SAM segmentation, meristem delineation, size and curvature estimation & signal quantification

### Necessary imports

In [ ]:
import os
from time import time as current_time

import numpy as np
import scipy.ndimage as nd
import pandas as pd
import matplotlib.pyplot as plt

import vtk

from timagetk.components import SpatialImage, LabelledImage
from timagetk.io import imread, imsave
from timagetk.plugins import h_transform, region_labeling, segmentation
from timagetk.plugins.resampling import isometric_resampling
from timagetk.plugins import labels_post_processing

from cellcomplex.utils.array_dict import array_dict

from cellcomplex.property_topomesh.creation import triangle_topomesh, vertex_topomesh
from cellcomplex.property_topomesh.analysis import compute_topomesh_property, compute_topomesh_vertex_property_from_faces, topomesh_property_median_filtering, topomesh_property_gaussian_filtering
from cellcomplex.property_topomesh.optimization import property_topomesh_vertices_deformation
from cellcomplex.property_topomesh.extraction import property_filtering_sub_topomesh, topomesh_connected_components
from cellcomplex.property_topomesh.io import read_ply_property_topomesh, save_ply_property_topomesh

from cellcomplex.property_topomesh.visualization.vtk_actor_topomesh import VtkActorTopomesh

from tissue_nukem_3d.microscopy_images.read_microscopy_image import read_czi_image, read_tiff_image
from tissue_nukem_3d.nuclei_mesh_tools import nuclei_image_surface_topomesh, up_facing_surface_topomesh

from tissue_analysis.property_spatial_image import PropertySpatialImage

from tissue_paredes.utils.tissue_image_tools import pseudo_gradient_norm

from visu_core.matplotlib import glasbey
from visu_core.vtk.display import vtk_display_notebook, vtk_save_screenshot_actors
from visu_core.vtk.volume import vtk_image_volume
from visu_core.vtk.actor import vtk_actor

from visu_core.vtk.utils.image_tools import vtk_image_data_from_image, image_to_vtk_cell_polydatas
from visu_core.vtk.utils.polydata_tools import vtk_combine_polydatas

### Import locally defined tools

In [ ]:
from utils.image_tools import labelled_image_projection
from utils.property_image_tools import image_property_morphological_operation, property_image_to_dataframe

### File path information

In [ ]:
 dirname = '/Users/gcerutti/Projects/RDP/LayerHeight_Marketa/test/'
#dirname = '/home/carlos/Documents/WORK/Image analysis/20160625 MS-E3 CLV-DR5/LD/RAW'

filename = 'E15_SAM02'
#filename = 'CLV3-CH-DR5-3VE-MS-E3-LD-SAM3'
#filename = 'CLV3-CH-DR5-3VE-MS-E3-LD-SAM5'

### Algorithm parameters

In [ ]:
# image parameters
#channel_names = ['PI']
channel_names = ['Signal','PI']
#channel_names = ['CLV3','DR5','PI']
membrane_channel = 'PI'
#signal_channels = ['CLV3','DR5']
signal_channels = ['Signal']
microscope_orientation = 1

# segmentation parameters
recompute_segmentation = False
gaussian_sigma = 0.75
h_min = 200
segmentation_gaussian_sigma = 0.5
volume_threshold = 100000

# SAM surface extraction parameters
recompute_surface = False
resampling_voxelsize = 1.
gaussian_curvature_threshold = -2.5e-4
min_curvature_threshold = -1e-3

# Signal quantification parameters
quantification_erosion_radius = 1.

# Visualization parameters
subsampling = 2
figure_size = 8

### Preparing folder architecture

In [ ]:
if not os.path.exists(dirname + "/" + filename + "/"):
    os.makedirs(dirname + "/" + filename + "/")

### Reading the microscopy image

In [ ]:
microscopy_filename = dirname + '/' + filename + '.czi'
img_dict = read_czi_image(microscopy_filename, channel_names)
if len(channel_names) == 1:
    img_dict = {channel_names[0]:img_dict}

In [ ]:
actors = []

image_data = vtk_image_data_from_image(img_dict[membrane_channel].get_array(), 
                                       voxelsize=img_dict[membrane_channel].voxelsize)
volume = vtk_image_volume(image_data,colormap='gray')
actors += [volume]

figure_filename = dirname + '/' + filename + '/' + filename + '_membrane_image.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Auto-seeded watershed segmentation

In [ ]:
segmentation_filename = dirname + '/' + filename + '/' + filename + '_' + membrane_channel + '_seg.tif'

if recompute_segmentation or not os.path.exists(segmentation_filename):
    img = img_dict[membrane_channel]
    voxelsize = np.array(img.voxelsize)

    smooth_image = nd.gaussian_filter(img, sigma=gaussian_sigma / voxelsize).astype(img.dtype)
    smooth_img = SpatialImage(smooth_image, voxelsize=voxelsize)

    ext_img = h_transform(smooth_img, h=h_min, method='min')

    seed_img = region_labeling(ext_img, low_threshold=1, high_threshold=h_min, method='connected_components')

    seg_smooth_image = nd.gaussian_filter(img, sigma=segmentation_gaussian_sigma / voxelsize).astype(img.dtype)
    seg_smooth_img = SpatialImage(seg_smooth_image, voxelsize=voxelsize)

    seg_img = segmentation(seg_smooth_img, seed_img, control='first', method='seeded_watershed')
    
    seg_volumes = dict(zip(np.arange(seg_img.max()) + 1, 
                           nd.sum(np.prod(voxelsize) * np.ones_like(seg_img),
                                  seg_img, 
                                  index = np.arange(seg_img.max())+1)))

    labels_to_remove = np.array(list(seg_volumes.keys()))[np.array(list(seg_volumes.values())) > volume_threshold]
    print("--> Removing too large labels :",labels_to_remove)
    for l in labels_to_remove:
        seg_img[seg_img == l] = 1
    
    imsave(segmentation_filename, seg_img)
else:
    seg_img = read_tiff_image(segmentation_filename)
    seg_img = LabelledImage(seg_img, no_label_id=0)


In [ ]:
actors = []

labels = [l for l in seg_img.labels() if l!=1]

cell_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                             labels=labels, 
                                             subsampling=[int(np.round(0.8/v)) for v in seg_img.voxelsize], 
                                             smoothing=10)
cell_polydata = vtk_combine_polydatas([cell_polydatas[l] for l in labels])
cell_actor = vtk_actor(cell_polydata,colormap='glasbey',value_range=(0,255))
actors += [cell_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_segmented_cells.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Surface extraction

In [ ]:
binary_img = SpatialImage(nd.gaussian_filter(255*(seg_img>1),sigma=1./np.array(seg_img.voxelsize)).astype(np.uint8),voxelsize=seg_img.voxelsize)

if resampling_voxelsize is not None:
    binary_img = isometric_resampling(binary_img, method=resampling_voxelsize, option='linear')
else:
    resampling_voxelsize = np.array(binary_img.voxelsize)

In [ ]:
surface_filename = dirname + '/' + filename + '/' + filename + '_' + membrane_channel + '_surface.ply'

curvature_properties = ['mean_curvature','gaussian_curvature','principal_curvature_min','principal_curvature_max']
    
if recompute_surface or not os.path.exists(surface_filename):
    topomesh = nuclei_image_surface_topomesh(binary_img,
                                             nuclei_sigma=resampling_voxelsize,
                                             density_voxelsize=resampling_voxelsize,
                                             maximal_length=10*resampling_voxelsize,
                                             intensity_threshold=64,
                                             decimation=100)

    topomesh = up_facing_surface_topomesh(topomesh, normal_method='orientation', upwards=microscope_orientation==1)

    start_time = current_time()
    print("--> Computing mesh curvature")
    compute_topomesh_vertex_property_from_faces(topomesh,'normal',neighborhood=3,adjacency_sigma=1.2)

    compute_topomesh_property(topomesh,'mean_curvature',2)
    for property_name in curvature_properties:
        compute_topomesh_vertex_property_from_faces(topomesh,property_name,neighborhood=3,adjacency_sigma=1.2)
    print("<-- Computing mesh curvature [", current_time() - start_time, "s]")

    topomesh.update_wisp_property('barycenter_z',0,dict(zip(topomesh.wisps(0),topomesh.wisp_property('barycenter',0).values(list(topomesh.wisps(0)))[:,2])))

    properties_to_save = {}
    properties_to_save[0] = curvature_properties + ['normal','barycenter_z']
    properties_to_save[1] = []
    properties_to_save[2] = curvature_properties + ['normal','area']
    properties_to_save[3] = []

    save_ply_property_topomesh(topomesh,surface_filename,properties_to_save)
else:
    topomesh = read_ply_property_topomesh(surface_filename)

In [ ]:
actors = []

surface_actor = VtkActorTopomesh(topomesh,2,property_name='principal_curvature_min',property_degree=0)
surface_actor.update(colormap='RdBu_r',value_range=(-0.1,0.1),opacity=1)
actors += [surface_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_surface_min_curvature.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(0,microscope_orientation,0))
vtk_display_notebook(actors)

### Curvature-based SAM delineation

In [ ]:
# convex_topomesh = property_filtering_sub_topomesh(topomesh,'gaussian_curvature',2,(gaussian_curvature_threshold,10))
convex_topomesh = property_filtering_sub_topomesh(topomesh,'principal_curvature_min',2,(min_curvature_threshold,10))
compute_topomesh_property(convex_topomesh,'area',2)

convex_topomeshes = topomesh_connected_components(convex_topomesh,degree=2)
mesh_scores = []
for i_mesh, mesh in enumerate(convex_topomeshes):
    mesh_area = mesh.wisp_property('area',2).values(list(mesh.wisps(2))).sum()
    mesh_center = mesh.wisp_property('barycenter',0).values(list(mesh.wisps(0))).mean(axis=0)
    
    mesh_scores += [np.sqrt(mesh_area) / np.linalg.norm((mesh_center - np.array(seg_img.extent)/2.)[:2])]
meristem_topomesh = convex_topomeshes[np.argmax(mesh_scores)]

meristem_area = meristem_topomesh.wisp_property('area',2).values(list(meristem_topomesh.wisps(2))).sum()
meristem_radius = np.sqrt(meristem_area/np.pi)

meristem_curvature = np.nanmedian(meristem_topomesh.wisp_property('mean_curvature',2).values(list(meristem_topomesh.wisps(2))))

In [ ]:
actors = []

# actor = VtkActorTopomesh(topomesh,2,property_name='gaussian_curvature',property_degree=0)
surface_actor = VtkActorTopomesh(topomesh,2,property_name='principal_curvature_min',property_degree=0)
# actor.update(colormap='RdBu_r',value_range=(-0.002,0.002))
surface_actor.update(colormap='RdBu_r',value_range=(-0.1,0.1),opacity=0.1)
# surface_actor.update(colormap='RdBu_r',value_range=(-0.1,0.1),opacity=1)
actors += [surface_actor]

# actor = VtkActorTopomesh(topomesh,2,property_name='gaussian_curvature',property_degree=0)
actor = VtkActorTopomesh(meristem_topomesh,2,property_name='principal_curvature_min',property_degree=0)
# actor.update(colormap='RdBu_r',value_range=(-0.002,0.002))
actor.update(colormap='RdBu_r',value_range=(-0.1,0.1))
actors += [actor]


figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_surface.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Meristem projection on the segmented image

In [ ]:
p_img = PropertySpatialImage(seg_img)
p_img.compute_image_property('layer')

In [ ]:
cell_center = p_img.image_property('barycenter')
cell_layer = p_img.image_property('layer')

cell_points = cell_center.values(p_img.labels)
meristem_points = meristem_topomesh.wisp_property('barycenter',0).values(list(meristem_topomesh.wisps(0)))
cell_meristem_distances = np.linalg.norm(cell_points[:,np.newaxis] - meristem_points[np.newaxis],axis=2).min(axis=1)
cell_meristem_distances = array_dict(cell_meristem_distances,keys=p_img.labels)

cell_meristem = {c:(cell_meristem_distances[c]<5.*resampling_voxelsize and cell_layer[c]==1) for c in p_img.labels}
p_img.update_image_property('meristem',cell_meristem)

image_property_morphological_operation(p_img,'meristem',method='closing',iterations=3)

cell_meristem = p_img.image_property('meristem')
for layer in [2,3]:
    layer_cells = np.array([l for l in p_img.labels if cell_layer[l]==layer])
    layer_neighbor_meristem_cells = [[n for n in p_img.image_graph.neighbors(l) if cell_meristem[n]] for l in layer_cells]
    layer_meristem_neighborhood_size = np.array(list(map(len,layer_neighbor_meristem_cells)))
    layer_meristem_cells = layer_cells[layer_meristem_neighborhood_size>3]
    for c in layer_meristem_cells:
        cell_meristem[c] = True
    p_img.update_image_property('meristem',cell_meristem)
    image_property_morphological_operation(p_img,'meristem',method='closing',iterations=3,layer_restriction=None)
    cell_meristem = p_img.image_property('meristem')

In [ ]:
actors = []

cell_meristem = p_img.image_property('meristem')

labels = [l for l in p_img.labels if p_img.image_property('barycenter')[l][0]>3*seg_img.extent[0]/5]

meristem_polydatas = image_to_vtk_cell_polydatas(seg_img,
                                                 labels=labels, 
                                                 cell_property=cell_meristem,
                                                 cell_polydatas=cell_polydatas)
meristem_polydata = vtk_combine_polydatas([meristem_polydatas[l] for l in labels])
meristem_actor = vtk_actor(meristem_polydata,colormap='jet',value_range=(0,4))
actors += [meristem_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cells.png'
#vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(1,0,0), view_up=(0,0,microscope_orientation))
vtk_display_notebook(actors)

### Meristem cells property computation

In [ ]:
p_img.compute_image_property('volume')

l1_meristem_cells = [c for c in p_img.labels if p_img.image_property('meristem')[c] and p_img.image_property('layer')[c]==1]
l1_meristem_cell_average_volume = np.nanmean([p_img.image_property('volume')[c] for c in l1_meristem_cells])

In [ ]:
l1_labels = [c for c in p_img.labels if p_img.image_property('layer')[c] == 1]
l1_cell_points = np.array([p_img.image_property('barycenter')[c] for c in l1_labels])
surface_points = topomesh.wisp_property('barycenter',0).values(list(topomesh.wisps(0)))
l1_surface_distances = np.linalg.norm(l1_cell_points[:,np.newaxis] - surface_points[np.newaxis],axis=2)
l1_cell_surface_vertex = np.array(list(topomesh.wisps(0)))[np.argmin(l1_surface_distances,axis=1)]

for property_name in curvature_properties:
    l1_cell_curvature = dict(zip(l1_labels,topomesh.wisp_property(property_name,0).values(l1_cell_surface_vertex)))
    p_img.update_image_property(property_name,{c:l1_cell_curvature[c] if c in l1_labels else np.nan for c in p_img.labels})

In [ ]:
from visu_core.vtk.utils.image_tools import image_to_surface_cell_scalar_property_polydata
from visu_core.vtk.utils.polydata_tools import vtk_tube_polydata
from visu_core.matplotlib.colormap import plain_colormap

actors = []

cell_curvature = p_img.image_property('mean_curvature')
curvature_polydatas = image_to_vtk_cell_polydatas(seg_img, 
                                                  labels=l1_labels, 
                                                  cell_polydatas=cell_polydatas, 
                                                  cell_property=cell_curvature)
curvature_polydata = vtk_combine_polydatas([curvature_polydatas[l] for l in l1_labels])
curvature_actor = vtk_actor(curvature_polydata,colormap='RdBu_r',value_range=(-0.1,0.1))
actors += [curvature_actor]

meristem_polydata = image_to_surface_cell_scalar_property_polydata(seg_img,cell_property=cell_meristem)

contour = vtk.vtkContourFilter()
contour.SetInputData(meristem_polydata)
contour.SetValue(0,1)
contour.SetGenerateTriangles(False)
contour.Update()
meristem_contour = vtk_tube_polydata(contour.GetOutput(),radius=0.5)

meristem_actor = vtk_actor(meristem_contour,colormap=plain_colormap('red'))
actors += [meristem_actor]

figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cell_curvature.png'
vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))
vtk_display_notebook(actors)

### Cell signal quantification

In [ ]:
if len(signal_channels) > 0:
    eroded_seg_img = labels_post_processing(seg_img, method='erosion', radius=quantification_erosion_radius, iterations=1)

    for i_s, signal_name in enumerate(signal_channels):
        signal_img = img_dict[signal_name]

        cell_total_signals = nd.sum(signal_img.get_array().astype(float),eroded_seg_img.get_array(),index=p_img.labels)
        cell_volumes = nd.sum(np.ones_like(eroded_seg_img.get_array()),eroded_seg_img.get_array(),index=p_img.labels)
        cell_signals = dict(zip(p_img.labels,cell_total_signals/cell_volumes))
        p_img.update_image_property(signal_name,cell_signals)

In [ ]:
if len(signal_channels) > 0:
    
    for i_s, signal_name in enumerate(signal_channels):
        actors = []

        signal_image_data = vtk_image_data_from_image(img_dict[signal_name].get_array(), 
                                                      voxelsize=img_dict[signal_name].voxelsize)
        signal_volume = vtk_image_volume(signal_image_data,colormap='gray')
        actors += [signal_volume]
        
        figure_filename = dirname + '/' + filename + '/' + filename + '_' + signal_name + '_signal_image.png'
        vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))

        cell_signal = p_img.image_property(signal_name)
        signal_polydatas = image_to_vtk_cell_polydatas(seg_img, 
                                                       labels=labels, 
                                                       cell_polydatas=cell_polydatas, 
                                                       cell_property=cell_signal)

        signal_polydata = vtk_combine_polydatas([signal_polydatas[l] for l in labels])
        signal_actor = vtk_actor(signal_polydata,colormap='gray')
        actors += [signal_actor]

        meristem_actor = vtk_actor(meristem_contour,colormap=plain_colormap('red'))
        actors += [meristem_actor]

        figure_filename = dirname + '/' + filename + '/' + filename + '_meristem_cell_' + signal_name + '_signal.png'
        vtk_save_screenshot_actors(actors, figure_filename, focal_point=(0,0,-microscope_orientation), view_up=(-microscope_orientation,0,0))

        vtk_display_notebook(actors)

### Cell data export

In [ ]:
cell_df = property_image_to_dataframe(p_img)

cell_data_filename = dirname + '/' + filename + '/' + filename + '_cell_data.csv'
cell_df.to_csv(cell_data_filename,index=False)

### SAM data export

In [ ]:
meristem_data = {}
meristem_data['filename'] = [filename]
meristem_data['area'] = [meristem_area]
meristem_data['mean_curvature'] = [meristem_curvature]
meristem_data['L1_cells'] = [len(l1_meristem_cells)]
meristem_data['L1_cell_volume'] = [l1_meristem_cell_average_volume]
meristem_df = pd.DataFrame().from_dict(meristem_data)

print(meristem_df)
meristem_data_filename = dirname + '/' + filename + '/' + filename + '_meristem_data.csv'
meristem_df.to_csv(meristem_data_filename,index=False)